# Training:

This project uses a machine learning model of classification and segmentation. It is based on a U-Net architecture, which uses a set of images and binary masks to train. 

### Build images and masks

The set of images and binary masks used for training can be generated via **satproc_extract_chips**.  
This tool uses an image (or a set of images) and a vectorial file to create another set of images of a specific size and their masks.
In addition, several features can be included like the rescale option to bring the output with pixel values between 0 to 255. All of them are fully explained below. 

The vector file contains many categories of objects, then the mask will have a binary band for each of them. 


![chips](img/mask_ej.png)

In [ ]:
!satproc_extract_chips     \
    /home/ro/incendios-forestales/data/img/dif_nbr.tif \
    -o /home/ro/incendios-forestales/data/data_train/160_80/ \
    --size 160 \    
    --step-size 80 \   
    --aoi /home/ro/incendios-forestales/data/shp/gt/fuego.geojson \
    --labels /home/ro/incendios-forestales/data/shp/gt/fuego.geojson \
    --label-property 'class' \
    --classes 'fire' \
    --rescale \
    --rescale-mode percentiles \
    --upper-cut 98 --lower-cut 2



In [6]:
#!ls /home/ro/incendios-forestales/data/data_train/160_80/

###### The options:

* **the argument** the path to the source image or a folder with contains a set of images

* **o** path to destiny images and masks
* **size**size of the generated images and masks 
* **step-size** the size of the step in the slide windows process. if the step size is smaller than the size then an overlap is generated. This could be useful to increase the amount of training data.
* **crs** it set an epsg in case the images don't have it. 
* **label-property** the name of the field in the shapefile where each category is defined. 
* **classes** name of the different categories (all split by a space)
* **aoi** path to the shapefile that delimits the region of interest. It can be the same that the shapefile with the categories or other.
* **rescale** to rescale the images to a range 1-255
* **rescale-mode** Rescaled mode:  percentiles, custom_rgb, values. 

Percentiles mode will rescale each band if the image to a scale defined by the percentiles considering the upper and lower cuts. For example --upper-cut 98 --lower-cut 2

The custom_rgb will rescale the 3 first bands from 0 to 0.3 and the rest of them considering the percentiles.

The values mode will rescale all the bands in the image considering the min and max values defined. For example --Min 0 --Max 0.3   


## Trainig  

First, we should import the libraries needed 

In [ ]:
from fire.unet.train import TrainConfig, train
from fire.unet.evaluate import plot_data_generator
import os

To set the parameters of the training process


In [ ]:
train_config = TrainConfig(width=320,  #  size of the image (160, 320,etc)
                     height=320,
                     n_channels=1,  #  number of channels of the image rgb -> 3
                     n_classes=1, # number of classes to clasify
                     apply_image_augmentation=True, #  increase the amount of training data with a image augmentation process
                     seed=42,
                     epochs=15, 
                     batch_size=16, 
                     steps_per_epoch=150, 
                     early_stopping_patience=3, 
                     validation_split=0.2, 
                     images_path=os.path.join('/home/ro/incendios-forestales/data/data_train_1D_diffNBR','S2_1class', '160_80'), #ruta a las imágenes
                     model_path=os.path.join('/home/ro/incendios-forestales/data/weights/', 'UNet_fire_1Dim_160_80_spe150.h5'),#  ruta al archivo de entrenamiento
                     evaluate=False,
                     weights = [0.5]) 

To plot some exaples of images and masks

In [ ]:

plot_data_generator(train_config=train_config, num_samples=3, fig_size=(30,20))

To train

In [ ]:
res_config = train(train_config)

Print some metrics

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

plt.subplot(121)
plt.plot(res_config.history['loss'])
plt.plot(res_config.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(122)
plt.plot(res_config.history['mean_iou'])
plt.plot(res_config.history['val_mean_iou'])
plt.title('mean_iou')
plt.ylabel('val_mean_iou')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')


plt.show()